In [8]:
from streamlit_app import *

In [10]:
import datetime
from logging import exception
from typing import List

import gspread
import numpy as np
import pandas as pd
import plotly.express as px
import streamlit as st
from google.oauth2 import service_account
import matplotlib.colors as mcolors

In [11]:
sheet_url = st.secrets["private_gsheets_url"]
df = get_worksheet(sheet_url, 0)

2022-03-03 23:59:12.477 
  command:

    streamlit run /home/wotashu/.pyenv/versions/3.9.10/lib/python3.9/site-packages/ipykernel_launcher.py [ARGUMENTS]


In [12]:
cheki_df = get_worksheet(sheet_url, 0)
idol_df = get_worksheet(sheet_url, 1)
venue_df = get_worksheet(sheet_url, 2)

In [16]:
venue_df[venue_df['location'].str.contains("PlatinumGarage")]

location neighborhod city prefecture country
74  PlatinumGarage西新宿スタジオ          本町  渋谷区        東京都      日本

In [77]:
cheki_df['cheki_id'] = cheki_df.index

In [78]:
name_cols = [i for i in cheki_df.columns if "Name" in i]

In [79]:
cheki_df['idol_count'] = cheki_df[name_cols].replace("", np.nan).count(axis=1)

In [80]:
cheki_df.replace("", np.nan, inplace=True)

In [81]:
merged_df = pd.merge(cheki_df, venue_df, how="left", left_on="Location", right_on="Location")

In [82]:
group_location = pd.DataFrame(merged_df.groupby("Location")["cheki_id"].count().sort_values(ascending=False)).reset_index()

In [83]:
group_location[:20]

Location  cheki_id
0           新宿HEADPOWER       489
1     Tokyo Link's Hall       404
2            Cosmic Lab       338
3                    郵送       227
4               池袋リヴォイス       160
5         池袋SOUND PEACE       159
6             Space Odd        94
7            Deseo Mini        85
8            赤羽レニーAlpha        78
9         Dr. Jeekhan's        74
10            Club Asia        57
11  Nagoya ReNY Limited        49
12          Batur Tokyo        47
13                五反田G+        47
14                   K2        47
15            新宿SUNFACE        43
16        Chelsea Hotel        42
17            1000 Club        41
18         AKIBA COLORS        41
19              大塚Deepa        39

In [84]:
group_location['Location']

0                    新宿HEADPOWER
1              Tokyo Link's Hall
2                     Cosmic Lab
3                             郵送
4                        池袋リヴォイス
                 ...            
168                KFC Akihabara
169    Bellesalle Shibuya Garden
170                       池袋AKビル
171         Zepp DiverCity Tokyo
172                     ベルサール西新宿
Name: Location, Length: 173, dtype: object

In [85]:
cheki_df.columns = [col.lower() for col in cheki_df.columns]

In [86]:
names = {}
name_cols = [col for col in cheki_df.columns if "name" in col]
for index, row in cheki_df.iterrows():
    names[index] = set([row[col] for col in name_cols if row[col] is not np.NaN])

In [87]:
cheki_df['names'] = cheki_df.apply(lambda x: [x[col] for col in name_cols if x[col] is not np.NaN], axis=1)

In [97]:
df2 = cheki_df[['cheki_id', 'names']]

In [98]:
df2

cheki_id                                              names
1            1  [矢島舞美@℃-ute, 中島早貴@℃-ute, 鈴木愛理@℃-ute, 岡井千聖@℃-ut...
2            2                                      [AIRI@Aither]
3            3                                    [SAKURA@Aither]
4            4                                     [YUUKA@Aither]
5            5                                     [木村美咲@ラストアイドル]
...        ...                                                ...
4439      4439                                             [Suzy]
4440      4440                                             [Suzy]
4441      4441                                             [Suzy]
4442      4442                                             [Suzy]
4443      4443                                             [Suzy]

[4443 rows x 2 columns]

In [109]:
df3 = pd.get_dummies(
    df2.explode('names'),
    columns=['names'],
    # prefix="", prefix_sep=""
).groupby('cheki_id').sum().reset_index()

In [110]:
new_df = pd.merge(cheki_df[['cheki_id','location', 'date']], df3, how='left', right_on='cheki_id', left_on='cheki_id')

In [116]:
pd.DataFrame(new_df[[col for col in new_df.columns if "names_" in col]].sum().sort_values(ascending=False))

0
names_Suzy             947
names_恵深あむ@メルティハート     404
names_瀬乃悠月@恒星宇宙        272
names_七瀬千夏@たまご姫【たまプリ】  272
names_楠木りほ@天使突抜ニ読ミ     217
...                    ...
names_古川未鈴@でんぱ組.inc      1
names_和ちひろ@忍者クライシス       1
names_土井千春@THAT's so     1
names_夏来彩世@プシュケとラーガ      1
names_宇咲ひかり@爆風もんす〜ん      1

[322 rows x 1 columns]

In [120]:
new_df.groupby(['cheki_id', 'location', 'date']).sum()

names_AIRI@Aither  \
cheki_id location                                 date                            
1        Bellesalle Shibuya Garden                2013-05-05                  0   
2        Best Western Plus Silverdale Beach Hotel 2014-06-21                  1   
3        Best Western Plus Silverdale Beach Hotel 2014-06-21                  0   
4        Best Western Plus Silverdale Beach Hotel 2014-06-21                  0   
5        Best Western Plus Silverdale Beach Hotel 2014-06-21                  0   
...                                                                         ...   
4439     Cosmic Lab                               2022-02-11                  0   
4440     Cosmic Lab                               2022-02-11                  0   
4441     Cosmic Lab                               2022-02-11                  0   
4442     Cosmic Lab                               2022-02-11                  0   
4443     Cosmic Lab                               2022-02-11                  0   

                                                              names_Aarie@ACT102  \
cheki_id location                                 date                             
1        Bellesalle Shibuya Garden                2013-05-05                   0   
2        Best Western Plus Silverdale Beach Hotel 2014-06-21                   0   
3        Best Western Plus Silverdale Beach Hotel 2014-06-21                   0   
4        Best Western Plus Silverdale Beach Hotel 2014-06-21                   0   
5        Best Western Plus Silverdale Beach Hotel 2014-06-21                   0   
...                                                                          ...   
4439     Cosmic Lab                               2022-02-11                   0   
4440     Cosmic Lab                               2022-02-11                   0   
4441     Cosmic Lab                               2022-02-11                   0   
4442     Cosmic Lab                               2022-02-11                   0   
4443     Cosmic Lab                               2022-02-11                   0   

                                                              names_Ale 愛礼@国際組  \
cheki_id location                                 date                           
1        Bellesalle Shibuya Garden                2013-05-05                 0   
2        Best Western Plus Silverdale Beach Hotel 2014-06-21                 0   
3        Best Western Plus Silverdale Beach Hotel 2014-06-21                 0   
4        Best Western Plus Silverdale Beach Hotel 2014-06-21                 0   
5        Best Western Plus Silverdale Beach Hotel 2014-06-21                 0   
...                                                                        ...   
4439     Cosmic Lab                               2022-02-11                 0   
4440     Cosmic Lab                               2022-02-11                 0   
4441     Cosmic Lab                               2022-02-11                 0   
4442     Cosmic Lab                               2022-02-11                 0   
4443     Cosmic Lab                               2022-02-11                 0   

                                                              names_Ally@Seishun Youth Academy  \
cheki_id location                                 date                                           
1        Bellesalle Shibuya Garden                2013-05-05                                 0   
2        Best Western Plus Silverdale Beach Hotel 2014-06-21                                 0   
3        Best Western Plus Silverdale Beach Hotel 2014-06-21                                 0   
4        Best Western Plus Silverdale Beach Hotel 2014-06-21                                 0   
5        Best Western Plus Silverdale Beach Hotel 2014-06-21                                 0   
...                                                                                        ...   
4439     Cosmic Lab                   